In [32]:
!pip install feedparser # RSS에서 XML 태그별 정보 추출
!pip install newspaper3k #인터넷신문기사 분석 (Ariticle() 사용하기 위한 목적)
!pip install konlpy # 우리말 형태소 분석 (주어진 문장에서 명사만 추출할 목적)
!pip install lxml[html_clean]

import feedparser
from newspaper import Article
from konlpy.tag import Okt

from collections import Counter

import hashlib # 해시 키 만들기 용

# from bs4 import BeautifulSoup # 명사 추출 후 출현 빈도 확인용(TF 구현용)

In [33]:
# 1. 모든 RSS 파일(xml 형식)을 돌아다니면서 기사의 제목/link를 추출
# urls에 우리가 검색할 RSS파일들의 주소를 list로
urls = ["https://rss.etnews.com/Section901.xml",
        "https://rss.etnews.com/Section902.xml",
        "https://rss.etnews.com/Section903.xml",
        "https://rss.etnews.com/Section904.xml",
        ]

# 아래 함수는 RSS 목록의 list인 urls를 받아서
# 해당 urls 내에 존재하는 title과 link를 추출
def crawl_rss(urls):
    array_rss = []  # title과 link가 들어갈 list
    seen_titles = set()  # 중복 검사를 위한 set

    for index_url in urls:  # urls 내의 url들을 순회
        print("[Crawl RSS]", index_url)
        parse_rss = feedparser.parse(index_url)  # 현재 xml 파싱 후 결과를 parse_rss에 저장

        for p in parse_rss.entries:  # parse 후 모든 entry 순회
            # title을 해시화한 후 중복 검사
            # 해시화 하는 건 그냥 평문보다는 이뻐서 했음
            title_hash = hashlib.sha256(p.title.encode()).hexdigest()
            if title_hash not in seen_titles:  # 중복되지 않은 경우만 추가
                seen_titles.add(title_hash)
                array_rss.append({'title': p.title, 'link': p.link})

    return array_rss


list_articls = crawl_rss(urls)
print(list_articls)

[Crawl RSS] https://rss.etnews.com/Section901.xml
[Crawl RSS] https://rss.etnews.com/Section902.xml
[Crawl RSS] https://rss.etnews.com/Section903.xml
[Crawl RSS] https://rss.etnews.com/Section904.xml
[{'title': '중국 스마트폰 제조사, 세계 시장 영향력 확대', 'link': 'https://www.etnews.com/20241105000259'}, {'title': '“우리 삶을 의미 있게 바꿀 소셜벤처 한자리에”…벤처기업協, 임팩트스타 데모데이', 'link': 'https://www.etnews.com/20241105000246'}, {'title': "보험설계사 '스카우트 비용' 조사해보니…수십억은 기본", 'link': 'https://www.etnews.com/20241105000181'}, {'title': "[홈쇼핑 송출수수료 해법은] 〈하·끝〉자율조정 '유명무실'…정부 적극 개입필요", 'link': 'https://www.etnews.com/20241105000222'}, {'title': '中 폴더블폰의 역습…폼팩터·자체 OS로 위협', 'link': 'https://www.etnews.com/20241105000218'}, {'title': "LG CNS, AI로 SW 개발하는 'AI MDD' 기술 개발", 'link': 'https://www.etnews.com/20241105000209'}, {'title': '“머스크가 만든 온라인 커뮤니티, 5.8만명 모여 美 대선 가짜뉴스 공유”', 'link': 'https://www.etnews.com/20241105000196'}, {'title': "비트코인 '활황'에 빗썸·업비트 주문·차트 기능 대폭 개선", 'link': 'https://www.etnews.com/20241105000184'}, {'title': '더존비즈

In [34]:
# RSS에서 추출한 기사 list에서 본문 title과 text를 추출
# 아래의 함수는 하나의 url을 입력 받아, 그 안의 title과 text를  추출하게 됨.
def crawl_article(url, languege='ko'):
    print("[Crawl Article]", url) # 현재 url
    a = Article(url, languege = languege) # Ariticle에 url과 언어옵션 지정
    a.download() # 해당 url의 기사 다운로드
    a.parse() # parsing!
    return a.title, a.text

print(crawl_article(list_articls[0]['link']))

[Crawl Article] https://www.etnews.com/20241105000259
('중국 스마트폰 제조사, 세계 시장 영향력 확대', "중국 스마트폰 제조사들의 영향권이 중국을 넘어 세계로 뻗고 있다. 주요 지역에서 스마트폰 시장 '빅2'인 삼성전자와 애플을 위협하고 있다. 세계 스마트폰 시장이 코로나19 이전으로 회복하고 있는 가운데 중국 제조사들의 영향력이 더욱 커질 것이라는 전망도 나온다.\n\n\n\n5일 시장조사업체 카운터포인트리서치에 따르면 올해 3분기 중국 제조사(샤오미·오포·비보 등)의 세계 스마트폰 시장 점유율은 32%다. 전년 동기 대비 2%포인트(P) 증가했다. 이 기간 샤오미와 비보는 각각 전년동기대비 1%p 오른 점유율인 14%, 9%를 기록했다. 오포는 8%를 유지했다.\n\n\n\n타룬 파탁 카운터포인트리서치 연구위원은 “샤오미는 강력한 성장력을 이어가며 4분기 연속 전년동기대비 판매량이 상승하며, 8월 판매량 2위를 차지했다”며 “비보는 3분기 상위 5개 업체 중 가장 빠른 성장세를 보이며 경쟁사들보다 우수한 성과를 거뒀다”고 분석했다. 이 기간 비보는 세계 최대 스마트폰 시장인 중국과 인도에서 판매량 1위를 기록한 것으로 알려졌다. 오포에 대해서는 “오포는 작년 3분기 이후 역대 가장 높은 판매량을 보였다”고 분석했다.\n\n\n\n중국 제조사들의 선전은 신흥국 중심 판매 전략 효과로 풀이된다. 이들은 '가성비'를 앞세워 중동·아프리카·라틴 아메리카 지역에서 크게 성장했다. 일부 지역에서는 저가형 제품과 프리미엄 제품을 함께 파는 믹스 전략을 선보이기도 했다. 특히 신흥국 판매 경로를 온라인보다 오프라인 위주로 구축했던 점이 판매량 증대로 이어졌다는 분석이다. 샤오미의 경우 중국을 제외한 지역에서 오프라인 매장에 더 공을 들이는 것으로 알려졌다.\n\n\n\n이 기간 빅2로 분류된 삼성전자와 애플은 점유율 확대에 실패했다. 삼성전자 스마트폰 판매량 점유율은 전년동기보다 1%p떨어진 19%, 애플은 전년 수준인 16

In [35]:
for article in list_articls:
    _, text = crawl_article(article['link'])
    article['text'] = text

print(list_articls[0]['text'])

[Crawl Article] https://www.etnews.com/20241105000259
[Crawl Article] https://www.etnews.com/20241105000246
[Crawl Article] https://www.etnews.com/20241105000181
[Crawl Article] https://www.etnews.com/20241105000222
[Crawl Article] https://www.etnews.com/20241105000218
[Crawl Article] https://www.etnews.com/20241105000209
[Crawl Article] https://www.etnews.com/20241105000196
[Crawl Article] https://www.etnews.com/20241105000184
[Crawl Article] https://www.etnews.com/20241105000162
[Crawl Article] https://www.etnews.com/20241105000148
[Crawl Article] https://www.etnews.com/20241105000129
[Crawl Article] https://www.etnews.com/20241105000128
[Crawl Article] https://www.etnews.com/20241105000114
[Crawl Article] https://www.etnews.com/20241105000044
[Crawl Article] https://www.etnews.com/20241104000440
[Crawl Article] https://www.etnews.com/20241105000013
[Crawl Article] https://www.etnews.com/20241104000364
[Crawl Article] https://www.etnews.com/20241104000315
[Crawl Article] https://www.

In [36]:
# 3. 긁어온 본문 test에서 명사 추출(키워드, 빈도수)
def get_keywords(text, nKeywords = 10):
    list_keywords = [] # 빈 키워드 리스트. 추후 append로 모든 기사에 대해 내용 추가

    spliter = Okt() # 문장을 형태소별로 쪼개는 spliter

    nouns = spliter.nouns(text) # noun을 추출하여 nouns에 저장
    count = Counter(nouns) # 각 명사의 수

    for n, c in count.most_common(nKeywords): # 가장 출현빈도 높은 명사부터 순차적으로 10번
        item = {'keyword' : n, 'count' : c} # 저장은 키워드, 카운트 형식으로
        list_keywords.append(item)

    return list_keywords

In [37]:
print(get_keywords(list_articls[0]['text']))

[{'keyword': '중국', 'count': 9}, {'keyword': '스마트폰', 'count': 9}, {'keyword': '조사', 'count': 7}, {'keyword': '시장', 'count': 7}, {'keyword': '판매량', 'count': 6}, {'keyword': '세계', 'count': 5}, {'keyword': '전자', 'count': 5}, {'keyword': '애플', 'count': 5}, {'keyword': '것', 'count': 5}, {'keyword': '년', 'count': 5}]


In [38]:
for article in list_articls:
    keywords = get_keywords(article['text'])
    article['keywords'] = keywords

print(list_articls[0])

{'title': '중국 스마트폰 제조사, 세계 시장 영향력 확대', 'link': 'https://www.etnews.com/20241105000259', 'text': "중국 스마트폰 제조사들의 영향권이 중국을 넘어 세계로 뻗고 있다. 주요 지역에서 스마트폰 시장 '빅2'인 삼성전자와 애플을 위협하고 있다. 세계 스마트폰 시장이 코로나19 이전으로 회복하고 있는 가운데 중국 제조사들의 영향력이 더욱 커질 것이라는 전망도 나온다.\n\n\n\n5일 시장조사업체 카운터포인트리서치에 따르면 올해 3분기 중국 제조사(샤오미·오포·비보 등)의 세계 스마트폰 시장 점유율은 32%다. 전년 동기 대비 2%포인트(P) 증가했다. 이 기간 샤오미와 비보는 각각 전년동기대비 1%p 오른 점유율인 14%, 9%를 기록했다. 오포는 8%를 유지했다.\n\n\n\n타룬 파탁 카운터포인트리서치 연구위원은 “샤오미는 강력한 성장력을 이어가며 4분기 연속 전년동기대비 판매량이 상승하며, 8월 판매량 2위를 차지했다”며 “비보는 3분기 상위 5개 업체 중 가장 빠른 성장세를 보이며 경쟁사들보다 우수한 성과를 거뒀다”고 분석했다. 이 기간 비보는 세계 최대 스마트폰 시장인 중국과 인도에서 판매량 1위를 기록한 것으로 알려졌다. 오포에 대해서는 “오포는 작년 3분기 이후 역대 가장 높은 판매량을 보였다”고 분석했다.\n\n\n\n중국 제조사들의 선전은 신흥국 중심 판매 전략 효과로 풀이된다. 이들은 '가성비'를 앞세워 중동·아프리카·라틴 아메리카 지역에서 크게 성장했다. 일부 지역에서는 저가형 제품과 프리미엄 제품을 함께 파는 믹스 전략을 선보이기도 했다. 특히 신흥국 판매 경로를 온라인보다 오프라인 위주로 구축했던 점이 판매량 증대로 이어졌다는 분석이다. 샤오미의 경우 중국을 제외한 지역에서 오프라인 매장에 더 공을 들이는 것으로 알려졌다.\n\n\n\n이 기간 빅2로 분류된 삼성전자와 애플은 점유율 확대에 실패했다. 삼성전자 스마트폰 판매량 점유율은 전년동기보다 1%p떨어진 19%, 애

In [39]:
# 검색어를 입력받아서, 해당 검색어를 가진 문서를 출력
query = input()

# 아래 함수는 쿼리를 입력받은 후, 정해진 문서의 키워드 리스트에서
# 쿼리가 그 키워드 중 하날호 존재라는지 검색
def search_articles(query, list_keywords):
    nWords = 0 # 쿼리의 출현 횟수
    for kw in list_keywords:
        if query == kw['keyword']: # 키워드가 있다면
            nWords = kw['count']
            break

    return nWords

# 문서전체 검색
for article in list_articls:
    nQuery = search_articles(query, article['keywords'])
    if nQuery != 0:
        print('[TF]', nQuery, '[Title]', article['title'])
        print('[URL]', article['link'])

중국
[TF] 9 [Title] 중국 스마트폰 제조사, 세계 시장 영향력 확대
[URL] https://www.etnews.com/20241105000259
[TF] 12 [Title] 中 폴더블폰의 역습…폼팩터·자체 OS로 위협
[URL] https://www.etnews.com/20241105000218
[TF] 5 [Title] “35세 이전에 결혼하면 30만원 드려요”… 中 결혼보조금에 네티즌 뿔났다
[URL] https://www.etnews.com/20241104000414
[TF] 5 [Title] “中 LCD 패널업체, 의도적인 생산 중단으로 가격 안정화”
[URL] https://www.etnews.com/20241105000249
[TF] 4 [Title] 후진해서 날개에 '착'… 中 플라잉카 모습은?
[URL] https://www.etnews.com/20241101000297
[TF] 8 [Title] 세계 2위 폭스바겐 위기…3위 현대차 추격에 '지각변동' 예고
[URL] https://www.etnews.com/20241101000194
